## Associated math

$$Xw=y$$  
where:  
$X$ - dataset with features  
$w$ - variables, we want to find (unknowns)  
$y$ - target vector

$$Xw=\hat{y}$$  
where:  
$X$ - dataset with features  
$w$ - variables, we want to find (unknowns)  
$\hat{y}$ - vector in $Col(X)$ closest to $y$

$$w=X^T(X^TX)^{-1}y$$  
where:  
$X^T$ - dataset with features transpose  
$X$ - dataset with features  
$w$ - variables, we want to find (unknowns)  
$y$ - target vector

$$X=QR$$


$$Q^TQ=I$$


$$X=\left[\begin{array}{ccccc}
\mid&\mid&\mid&\cdots&\mid\\
x_1&x_2&x_3&\cdots&x_n\\
\mid&\mid&\mid&\cdots&\mid\\
\end{array}\right]\in \mathbb{R}^{mxn}$$

$$X_1=x_1$$

$$ X_2 = x_2 - \frac{X_1^Tx_2}{X_1^TX_1}X_1$$

$$ X_3 = x_3 - \frac{X_1^Tx_3}{X_1^TX_1}X_1- \frac{X_2^Tx_3}{X_2^TX_2}X_2$$

$$ X_n = x_n - \frac{X_1^Tx_n}{X_1^TX_1}X_1- \frac{X_2^Tx_n}{X_2^TX_2}X_2-\cdots -\frac{X_{n-1}^Tx_n}{X_{n-1}^TX_{n-1}}X_{n-1}$$

$$q_i = \frac{X_i}{||X_i||_2}\quad$$  
for $i = 1,2,3,\cdots,n$

$$Q=\left[\begin{array}{ccccc}
\mid&\mid&\mid&\cdots&\mid\\
q_1&q_2&q_3&\cdots&q_n\\
\mid&\mid&\mid&\cdots&\mid\\
\end{array}\right]\in \mathbb{R}^{mxn}$$


$$Q^TX=Q^TQR$$


$$Q^TX=IR$$


$$R=Q^TX$$

$$X^TXw=X^Ty$$  
where:  
$X$ - dataset with features  
$w$ - variables, we want to find (unknowns)  
$y$ - target vector

$$(QR)^TQRw=(QR)^Ty$$  

$$R^TQ^TQRw=R^TQ^Ty$$  

$$Rw=Q^Ty$$  

$$w=R^{-1}Q^Ty$$  

In [17]:
import numpy as np

In [90]:
matrix = np.array([2,3,7,2,0,1,7,3,5,4,8,6,6,8,6,1,6,8,5,6]).reshape(4,5).T

### QR factorization function

In [73]:
def q_r_factorization(matrix):
    '''
    This function factors an input matrix into two matrices: Q and R
    
    INPUT:
        matrix: array_like - mxn matrix
    OUTPUT:
        Q: ndarray of float - matrix with orthonormal columns of shape mxn
        R: ndarray of float - upper triangular square matrix of shape nxn 
    '''
    matrix = matrix.astype(float)
    Q = np.zeros(matrix.shape)
    A = matrix.copy()
    for i in range(A.shape[1]):
        Q[:,i] = A[:,0]/np.linalg.norm(A[:,0])   
        for j in range(1, A.shape[1]):
            A[:,j] = A[:,j] - np.dot(A[:,0], A[:,j])/np.dot(A[:,0], A[:,0])*A[:,0]    
        A = A[:,1:]

    R = np.zeros((Q.shape[1], Q.shape[1]))
    for i in range(R.shape[0]):
        for j in range(R.shape[0]):
            if i>j:
                R[i,j]=0
            else:
                R[i,j] = np.dot(Q[:,i].T,matrix[:,j])
    return Q, R

In [70]:
matrix = np.array([2,3,7,2,0,1,7,3,5,4,8,6,6,8,6,1,6,8,5,6]).reshape(4,5).T
Q,R = q_r_factorization(matrix)
print('Original Matrix')
print('-----------------------')
print(matrix)
print('\nQ - matrix with orthonormal column')
print('-----------------------')
print(np.around(Q,2))
print('\nUpper triangular square matrix R')
print('-----------------------')
print(np.around(R,2))

Original Matrix
-----------------------
[[2 1 8 1]
 [3 7 6 6]
 [7 3 6 8]
 [2 5 8 5]
 [0 4 6 6]]

Q - matrix with orthonormal column
-----------------------
[[ 0.25 -0.09  0.83 -0.36]
 [ 0.37  0.61 -0.4  -0.43]
 [ 0.86 -0.37 -0.14  0.32]
 [ 0.25  0.45  0.25 -0.12]
 [ 0.    0.54  0.27  0.75]]

Upper triangular square matrix R
-----------------------
[[ 8.12  6.65 11.32 10.59]
 [ 0.    7.47  7.59  6.11]
 [ 0.    0.    7.08  0.25]
 [ 0.    0.    0.    3.55]]


In [88]:
import numpy as np
class LinearRegression:
    
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y):
        
        
        X = X.astype(float)
        y = y.astype(float)
        
        if self.fit_intercept == True:

            X_average = np.mean(X, axis=0)
            X = X - X_average
            y_average = np.mean(y)
            y = y - y_average
            # Here I put q_r factorization function:
            Q,R = q_r_factorization(X)
            R_invQ_T = np.dot(np.linalg.inv(R), Q.T)
            self.coef = np.dot(R_invQ_T, y)
            self.intercept = y_average - np.dot(X_average, self.coef.T)
        else:
            # Here I put q_r factorization function:
            Q, R = q_r_factorization(X)
            R_invQ_T = np.dot(np.linalg.inv(R), Q.T)
            self.coef = np.dot(R_invQ_T, y)
            self.intercept = 0
         
        return self
    
    def predict(self, X):
        
        y_pred = np.dot(X, self.coef.T) + self.intercept    
        return y_pred
    
    def score(self, X, y):
        
        y_pred = self.predict(X)
        numerator = ((y - y_pred)**2).sum()
        denominator = ((y - np.mean(y))**2).sum()
        score = 1 - (numerator/denominator)
        return score

In [89]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
X, y = datasets.load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)
reg = LinearRegression(fit_intercept=True)
reg.fit(X_train,y_train)
r2 = reg.score(X_train, y_train)
r2_test = reg.score(X_test, y_test)
print('Coefficient of determination\nTraining set')
print('--------------------------')
print(r2)
print('__________________________')
print('Coefficient of determination\nTesting set')
print('--------------------------')
print(r2_test)

Coefficient of determination
Training set
--------------------------
0.5150677275847442
__________________________
Coefficient of determination
Testing set
--------------------------
0.5164196511816805
